In [0]:
USE CATALOG claims360_dev;

-- Alert: total denied claims in last 24h > threshold
WITH params AS (
  SELECT 25 AS threshold
),
claims_denied_last24 AS (
  SELECT
    payer_name,
    COUNT(*) AS denied_claims_24h
  FROM gold.fact_claim
  WHERE has_denial_any = 1
    AND GREATEST(
          COALESCE(last_835_event_ts, TIMESTAMP '1900-01-01'),
          COALESCE(last_277_event_ts, TIMESTAMP '1900-01-01')
        ) >= CURRENT_TIMESTAMP() - INTERVAL 24 HOURS
  GROUP BY payer_name
),
totals AS (
  SELECT COALESCE(SUM(denied_claims_24h), 0) AS denied_claims_24h
  FROM claims_denied_last24
)
SELECT denied_claims_24h,
       (SELECT threshold FROM params) AS threshold
FROM totals
WHERE denied_claims_24h > (SELECT threshold FROM params);